In [1]:
from sympy import derive_by_array
from sympy import symbols, sin, cos, Inverse, Matrix
from sympy import simplify

In [2]:
t,x,y,z = symbols("t x y z")
r,theta,phi = symbols("r theta phi")
M = symbols("M")

In [3]:
cartesianPartialPolar = derive_by_array([t, r*cos(theta), r * sin(theta)*sin(phi), r*sin(theta)*cos(phi)], [t, r, theta,phi]) # partial x / partial theta
polarPartialCartesian = Inverse(Matrix(cartesianPartialPolar))

In [4]:
cartesianPartialPolar

[[1, 0, 0, 0], [0, cos(theta), sin(phi)*sin(theta), sin(theta)*cos(phi)], [0, -r*sin(theta), r*sin(phi)*cos(theta), r*cos(phi)*cos(theta)], [0, 0, r*sin(theta)*cos(phi), -r*sin(phi)*sin(theta)]]

In [42]:
A = cartesianPartialPolar.tomatrix().inv()

In [56]:
polarPartialCartesian = Array(simplify(A))

In [57]:
type(polarPartialCartesian)

sympy.tensor.array.dense_ndim_array.ImmutableDenseNDimArray

In [61]:
schwarzschild = Array(diag(-(1-2*M/r), 1/(1-2*M/r), r**2, r**2 * sin(theta)**2))
schwarzschild

[[2*M/r - 1, 0, 0, 0], [0, 1/(-2*M/r + 1), 0, 0], [0, 0, r**2, 0], [0, 0, 0, r**2*sin(theta)**2]]

In [62]:
from sympy import tensorproduct
from sympy import tensorcontraction

In [63]:
g = tensorproduct(schwarzschild, polarPartialCartesian, polarPartialCartesian)

In [66]:
C = tensorcontraction(g,(0,2))
D = tensorcontraction(C,(1,2))

In [68]:
simplify(D)

[[(2*M - r)/r, 0, 0, 0], [0, (-r*cos(theta)**2 + sin(phi)*sin(theta)**2)/(2*M - r), (r + sin(phi))*sin(theta)*cos(theta)/(r*(2*M - r)), cos(phi)/(r*(2*M - r))], [0, r*(r*(cos(phi - 2*theta) - cos(phi + 2*theta))/4 + sin(phi)**2*sin(theta)*cos(theta) + cos(phi)**2), -r*sin(phi)*sin(theta)**2 + sin(phi)**2*cos(theta)**2 + cos(phi)**2/tan(theta), (sin(2*theta)/2 - 1)*sin(phi)*cos(phi)/sin(theta)**2], [0, r*(r*sin(2*theta)/2 - sin(phi) + cos(phi - 2*theta)/4 - cos(phi + 2*theta)/4)*sin(theta)**2*cos(phi), -(r*sin(theta)**3 + sin(phi)*sin(theta)**3 + sqrt(2)*sin(phi)*cos(theta + pi/4))*sin(theta)*cos(phi), sin(phi)**2 + sin(theta)*cos(phi)**2*cos(theta)]]

Below is existing code that we know works.
Computing some simple metric connections and curvature

In [5]:
from sympy.tensor.tensor import TensorIndexType, TensorHead, tensor_indices
from sympy.tensor.toperators import PartialDerivative
from sympy import symbols, Rational, sin, cos
Lorentz = TensorIndexType("Lorentz")
x = TensorHead("x", [Lorentz]) # coordinates
g = TensorHead("g", [Lorentz,Lorentz]) # metric
mu, nu, rho, sigma = tensor_indices("mu nu rho sigma ", Lorentz)

In [6]:
Christoffel = Rational(1,2) * (
    g(sigma,rho) * PartialDerivative(g(-sigma,-mu), x(nu))
    + g(sigma,rho) * PartialDerivative(g(-sigma,-nu), x(mu))
    - g(sigma,rho) * PartialDerivative(g(-mu,-nu), x(sigma)))
Christoffel

(1/2)*(g(L_0, rho)*PartialDerivative(g(-L_0, -mu), x(nu)) + g(L_0, rho)*PartialDerivative(g(-L_0, -nu), x(mu)) + (-1)*g(L_0, rho)*PartialDerivative(g(-mu, -nu), x(L_0)))

In [7]:
g(mu,nu).replace_with_arrays({g(mu,nu): [[1,0],[0,1]]})

[[1, 0], [0, 1]]

In [8]:
t,r,theta,phi,M = symbols("t r theta phi M")

repl = {
    g(-mu,-nu): [[1,0],[0,r**(2)]],
    x(mu): [r,theta],
    Lorentz: [[1,0],[0,r**(2)]]
}

repl = {
    g(-mu,-nu): [[-(1-(2*M)/r),0,0,0],
                 [0,1/(1-(2*M)/r),0,0],
                 [0,0,r**2,0],
                 [0,0,0,r**2 * sin(theta)**2]],
    x(mu): [t,r,theta,phi],
    Lorentz: [[-(1-(2*M)/r),0,0,0],
        [0,1/(1-(2*M)/r),0,0],
        [0,0,r**2,0],
        [0,0,0,r**2 * sin(theta)**2]]
}
g(-mu,-nu).replace_with_arrays(repl)

[[2*M/r - 1, 0, 0, 0], [0, 1/(-2*M/r + 1), 0, 0], [0, 0, r**2, 0], [0, 0, 0, r**2*sin(theta)**2]]

In [9]:
Gamma = Christoffel.replace_with_arrays(repl)

In [ ]:
Gamma

[[[0, -M*(2*M/r - 1)/(-2*M + r)**2, 0, 0], [-M*(2*M/r - 1)/(-2*M + r)**2, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[M*(-2*M + r)**2/(r**4*(-2*M/r + 1)), 0, 0, 0], [0, -M*(-2*M + r)**2/(r**4*(-2*M/r + 1)**3), 0, 0], [0, 0, -(-2*M + r)**2/(r*(-2*M/r + 1)), 0], [0, 0, 0, -(-2*M + r)**2*sin(theta)**2/(r*(-2*M/r + 1))]], [[0, 0, 0, 0], [0, 0, 1/r, 0], [0, 1/r, 0, 0], [0, 0, 0, -sin(theta)*cos(theta)]], [[0, 0, 0, 0], [0, 0, 0, 1/r], [0, 0, 0, cos(theta)/sin(theta)], [0, 1/r, cos(theta)/sin(theta), 0]]]

diff geom